# Label side

In [1]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-03-20"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape

(2743200, 6)

In [2]:
np.save("data/btc_1m.npy", trading_1m)

In [1]:
import numpy as np

trading_1m = np.load("data/btc_1m.npy")
trading_1m.shape

(2743200, 6)

In [2]:
from custom_indicators.toolbox.bar.dollar_bar import estimate_dollar_bar_threshold

for i in [90, 60 * 6, 60 * 24]:
    threshold = estimate_dollar_bar_threshold(trading_1m, i)
    print(f"{i}m: {threshold}")

90m: 789511902.1743338
360m: 3216529968.747286
1440m: 12866119871.989143


In [3]:
from jesse import helpers

from custom_indicators.toolbox.bar.dollar_bar import build_dollar_bar
from model.config import DOLLAR_BAR_THRESHOLD_MID

print(DOLLAR_BAR_THRESHOLD_MID)

tp = helpers.date_to_timestamp("2025-03-01")

label_bar = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_MID)

print(label_bar.shape)

3216529968.747286
(7578, 6)


In [4]:
from custom_indicators.toolbox.labeler.zigzag_labeler import zigzag

res = zigzag(label_bar[:, 2], 0.08)
np.unique(res, return_counts=True)

(array([-1,  1]), array([3287, 4291]))

In [5]:
label = np.column_stack((label_bar, res))
label

array([[1.57793886e+12, 7.15598000e+03, 7.44700000e+03, ...,
        6.86344000e+03, 4.44652267e+05, 1.00000000e+00],
       [1.57820682e+12, 7.44720000e+03, 7.93262000e+03, ...,
        7.30300000e+03, 4.19224785e+05, 1.00000000e+00],
       [1.57841844e+12, 7.93264000e+03, 7.96557000e+03, ...,
        7.87011000e+03, 3.91941223e+05, 1.00000000e+00],
       ...,
       [1.74240630e+12, 8.39378000e+04, 8.49038000e+04, ...,
        8.35000000e+04, 3.86130120e+04, 1.00000000e+00],
       [1.74241038e+12, 8.49038000e+04, 8.51798000e+04, ...,
        8.48776000e+04, 4.12047380e+04, 1.00000000e+00],
       [1.74241428e+12, 8.51798000e+04, 8.68098000e+04, ...,
        8.51639000e+04, 3.75052050e+04, 1.00000000e+00]])

In [6]:
np.save("data/label_side.npy", label[label[:, 0] <= tp])
print(label.shape)
print(label[label[:, 0] <= tp].shape)

(7578, 7)
(7442, 7)


In [7]:
from jesse import utils

df_dollar_bar = utils.numpy_candles_to_dataframe(
    label_bar[label_bar[:, 0] <= tp], name_date="time"
)
df_dollar_bar["zigzag"] = res[label_bar[:, 0] <= tp]
df_dollar_bar.head(1)

,time,open,close,high,low,volume,zigzag
2020-01-02 04:21:00,2020-01-02 04:21:00,7155.98,7447.0,7495.0,6863.44,444652.267,1


In [8]:
from lightweight_charts import Chart

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_dollar_bar[["time", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name="zigzag", style="dotted")
line.set(df_dollar_bar[["time", "zigzag"]])

chart.show(block=True)

2025-03-26 21:24:22.431 python[66557:13784911] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-26 21:24:22.431 python[66557:13784911] +[IMKInputSession subclass]: chose IMKInputSession_Modern


RuntimeError: asyncio.run() cannot be called from a running event loop

# Label meta

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

label_side_1h = np.load("data/label_side.npy")
print(label_side_1h.shape)
print(np.unique(label_side_1h[:, 6], return_counts=True))

df_features = pd.read_parquet("data/features_6h.parquet")
print(df_features.shape)

side_model = lgb.Booster(model_file="model/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob
side_model_pred = np.where(side_model_pred_prob > 0.5, 1, -1)
print(np.unique(side_model_pred, return_counts=True))

(7442, 7)
(array([-1.,  1.]), array([3231, 4211]))
(7442, 11841)
(array([-1,  1]), array([3295, 4147]))


In [3]:
meta_label = (side_model_pred.astype(int) == label_side_1h[:, 6].astype(int)).astype(
    int
)
np.unique(meta_label, return_counts=True)

(array([0, 1]), array([1092, 6350]))

In [4]:
label_meta_1h = label_side_1h.copy()
label_meta_1h[:, 6] = meta_label
np.save("data/label_meta.npy", label_meta_1h)